# IoT Malware Binary Classification Prediction

This notebook focuses on loading a pre-trained binary classification model (Malware vs. Benign), loading test data, and using the model to make and visualize binary predictions.

In [ ]:
# ==============================================================================
# CORE SCRIPT: IoT Malware Binary Classification Prediction
# ==============================================================================

# --- Step 1: Remove Google Drive mounting as this script is for Raspberry Pi ---
# from google.colab import drive
# drive.mount('/content/drive') # This line is commented out for Raspberry Pi
print("✅ Script is configured for Raspberry Pi execution.")

Mounted at /content/drive
✅ Google Drive mounted successfully.


In [ ]:
# --- Step 2: Import all necessary libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, accuracy_score # Import necessary metrics for evaluation

## Define Binary Model Paths

Define the file paths for your saved binary classification model and its corresponding scaler.

In [ ]:
# --- Step 3: Define the paths to your saved binary model and scaler ---
# IMPORTANT: Update these paths to where your binary model and scaler are saved
# These paths should be absolute or relative paths on your Raspberry Pi
BINARY_MODEL_PATH = '/home/pi/Desktop/iot/model/iot_malware_svm_subset_model.pkl' # Example path, UPDATE THIS PATH
BINARY_SCALER_PATH = '/home/pi/Desktop/iot/model/iot_malware_svm_subset_scaler.pkl' # Example path, UPDATE THIS PATH

print(f"Binary Model Path: {BINARY_MODEL_PATH}")
print(f"Binary Scaler Path: {BINARY_SCALER_PATH}")

Binary Model Path: /content/drive/MyDrive/Colab Notebooks/ML_Models/3_SVM_WORKING/Model_V1/iot_malware_svm_subset_model.pkl
Binary Scaler Path: /content/drive/MyDrive/Colab Notebooks/ML_Models/3_SVM_WORKING/Model_V1/iot_malware_svm_subset_scaler.pkl


## Load Binary Model and Scaler

Load the saved binary classification model and scaler from the defined paths.

In [ ]:
# --- Step 4: Load the saved binary model and scaler ---
print("\n" + "="*50)
print("LOADING BINARY MODEL AND SCALER")
print("="*50)

try:
    # Load the model
    loaded_binary_model = joblib.load(BINARY_MODEL_PATH)
    # Load the scaler
    loaded_binary_scaler = joblib.load(BINARY_SCALER_PATH)
    print("\nBinary model and scaler loaded successfully.")

except FileNotFoundError:
    print("\nError: Binary model or scaler file not found.")
    print(f"Please check the paths: \nModel: {BINARY_MODEL_PATH} \nScaler: {BINARY_SCALER_PATH}")
    loaded_binary_model = None # Set to None if loading fails
    loaded_binary_scaler = None # Set to None if loading fails
except Exception as e:
    print(f"\nAn error occurred during binary model/scaler loading: {e}")
    loaded_binary_model = None # Set to None if loading fails
    loaded_binary_scaler = None # Set to None if loading fails

# Check if model and scaler were loaded successfully before proceeding
if loaded_binary_model is None or loaded_binary_scaler is None:
    raise Exception("Binary Model or Scaler failed to load. Please fix the paths and try again.")


LOADING BINARY MODEL AND SCALER

Binary model and scaler loaded successfully.


## Load and Combine Test Data

Load and combine all CSV files from the test data directory.

In [ ]:
# --- Step 5: Load and combine all test data ---
DATASET_ROOT_PATH = '/home/pi/Desktop/iot/test_data' # Set the path to your dataset on Raspberry Pi

print("\nLoading and combining all data files...")
all_dfs = []

all_data_files = []
for dirpath, _, filenames in os.walk(DATASET_ROOT_PATH):
    for f in filenames:
        if f.endswith('.csv'):
            all_data_files.append(os.path.join(dirpath, f))

if not all_data_files:
    raise Exception(f"No CSV files found in the specified path: {DATASET_ROOT_PATH}")


for file_path in all_data_files:
    try:
        # Load each CSV file
        df = pd.read_csv(file_path)
        all_dfs.append(df)
        # print(f"  Successfully loaded {file_path}") # Uncomment for detailed progress
    except Exception as e:
        print(f"  Error loading {file_path}: {e}")

# Combine all DataFrames
if all_dfs:
    combined_test_df_binary = pd.concat(all_dfs, ignore_index=True)
    print("\nData combining complete.")
    print(f"Shape of combined test dataset: {combined_test_df_binary.shape}")
    # Display the first few rows and info to understand the structure
    print("\nFirst 5 rows of the combined test dataset:")
    print(combined_test_df_binary.head())
    print("\nInfo of the combined test dataset:")
    combined_test_df_binary.info()

else:
    raise Exception("No data files were successfully loaded from the test directory.")


Loading and combining all data files...


In [ ]:
# # --- Step 5: Load and combine a subset of test data ---
# # This cell is commented out as it was for Colab environment
# # import os # Import os here
# # import pandas as pd # Import pandas here
# # DATASET_ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/test_data' # Set the path to your dataset
# # ROWS_TO_LOAD_PER_FILE = 1000 # Define how many rows to load from each CSV file. Adjust this number based on your RAM.

# # print("\nLoading and combining a subset of data files...")
# # all_dfs = []

# # all_data_files = []
# # for dirpath, _, filenames in os.walk(DATASET_ROOT_PATH):
# #     for f in filenames:
# #         if f.endswith('.csv'):
# #             all_data_files.append(os.path.join(dirpath, f))

# # if not all_data_files:
# #     raise Exception(f"No CSV files found in the specified path: {DATASET_ROOT_PATH}")


# # for file_path in all_data_files:
# #     try:
# #         # Load a subset of rows from each CSV file
# #         # Use nrows parameter to limit the number of rows read
# #         df = pd.read_csv(file_path, nrows=ROWS_TO_LOAD_PER_FILE)
# #         all_dfs.append(df)
# #         print(f"  Successfully loaded {len(df)} rows from {file_path}") # Print actual number of rows loaded
# #     except Exception as e:
# #         print(f"  Error loading {file_path}: {e}")

# # # Combine all DataFrames from the subsets
# # if all_dfs:
# #     combined_test_df_binary = pd.concat(all_dfs, ignore_index=True)
# #     print("\nData combining complete.")
# #     print(f"Shape of combined test dataset (subset): {combined_test_df_binary.shape}")
# #     # Display the first few rows and info to understand the structure
# #     print("\nFirst 5 rows of the combined test dataset (subset):")
# #     display(combined_test_df_binary.head())
# #     print("\nInfo of the combined test dataset (subset):")
# #     combined_test_df_binary.info()

# # else:
# #     raise Exception("No data files were successfully loaded from the test directory.")

## Prepare Data for Binary Prediction

Preprocess the combined test data to match the features expected by the loaded binary model and scaler.

In [ ]:
# --- Step 6: Prepare data for binary prediction ---

# Separate features (X) from potential identifier/label columns
# Assuming 'Flow_ID' and potentially 'label', 'device', 'attack_type'
# are not features used by the binary model
columns_to_drop_binary_prediction = ['Flow_ID', 'label', 'device', 'attack_type']

X_test_raw_binary = combined_test_df_binary.drop(columns=[col for col in columns_to_drop_binary_prediction if col in combined_test_df_binary.columns])

print(f"\nShape of raw test features (X_test_raw_binary): {X_test_raw_binary.shape}")

# Align columns with the scaler's expected features
# Get the feature names the scaler was fitted on from the loaded binary scaler
if 'loaded_binary_scaler' not in locals() or loaded_binary_scaler is None:
     raise Exception("Error: Binary scaler not loaded. Cannot align features.")

scaler_feature_names_binary = loaded_binary_scaler.feature_names_in_

# Add missing columns to X_test_raw_binary with default value (e.g., 0)
missing_cols_test_binary = set(scaler_feature_names_binary) - set(X_test_raw_binary.columns)
for c in missing_cols_test_binary:
    X_test_raw_binary[c] = 0

# Ensure the order of columns in X_test_raw_binary is the same as in scaler_feature_names_binary
X_test_aligned_binary = X_test_raw_binary[scaler_feature_names_binary]

print(f"Shape of aligned test data (X_test_aligned_binary): {X_test_aligned_binary.shape}")

# Scale the aligned test data using the loaded binary scaler
X_test_scaled_binary = loaded_binary_scaler.transform(X_test_aligned_binary)
print("\nTest data scaled successfully using the loaded binary scaler.")

## Make Binary Predictions

Use the loaded binary model and the prepared test data to make predictions (Malware or Benign).

In [ ]:
# --- Step 7: Make binary predictions ---

# Make predictions on the scaled test data
if 'loaded_binary_model' in locals() and loaded_binary_model is not None:
    binary_predictions = loaded_binary_model.predict(X_test_scaled_binary)
    print("\nBinary predictions made on the test data.")

    # Display the predictions (0: Benign, 1: Malware)
    print("\nSample Binary Predictions (0: Benign, 1: Malware):")
    print(binary_predictions[:20]) # Display first 20 predictions

    # Count the number of benign and malware predictions
    predicted_binary_counts = pd.Series(binary_predictions).value_counts().sort_index()
    print("\nPredicted Binary Counts (0: Benign, 1: Malware):")
    print(predicted_binary_counts)

else:
    raise Exception("Error: Binary model not loaded. Cannot make predictions.")

## Visualize Binary Predictions

Visualize the distribution of the predicted binary labels (Malware vs. Benign).

In [ ]:
# --- Step 8: Visualize binary predictions ---

# Map encoded predictions back to original labels
binary_labels = ['Benign', 'Malware']
predicted_binary_labels = [binary_labels[pred] for pred in binary_predictions]

# Calculate the counts of each predicted binary class
predicted_binary_counts_labels = pd.Series(predicted_binary_labels).value_counts()

# Define the save directory
# This path should be an absolute or relative path on your Raspberry Pi
PLOTS_SAVE_FOLDER = '/home/pi/Desktop/iot/test_output/plots/'
os.makedirs(PLOTS_SAVE_FOLDER, exist_ok=True) # Create the directory if it doesn't exist

# Create and save the bar plot
plt.figure(figsize=(8, 6))
ax = sns.barplot(x=predicted_binary_counts_labels.index, y=predicted_binary_counts_labels.values, palette='viridis')

# Add title and labels
plt.title('Distribution of Predicted Binary Classes (Malware vs. Benign)')
plt.xlabel('Predicted Class')
plt.ylabel('Count')

# Add text labels on top of the bars
for container in ax.containers:
    ax.bar_label(container)

# Save the plot
barplot_filepath = os.path.join(PLOTS_SAVE_FOLDER, 'predicted_binary_class_distribution.png')
plt.tight_layout()
plt.savefig(barplot_filepath)
print(f"Saved predicted class distribution plot to {barplot_filepath}")
plt.close() # Close the plot figure to free memory


# Optional: If the combined_test_df_binary has a 'label' column with true binary labels (0 or 1),
# you can generate evaluation metrics and a confusion matrix.
if 'label' in combined_test_df_binary.columns:
    print("\n" + "="*50)
    print("BINARY MODEL EVALUATION (if true labels available)")
    print("="*50)

    y_true_binary = combined_test_df_binary['label']

    # Calculate and display Accuracy
    accuracy = accuracy_score(y_true_binary, binary_predictions)
    print(f"\nAccuracy: {accuracy:.4f}")

    # Calculate and display Confusion Matrix
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true_binary, binary_predictions)
    print(cm)

    # Display Classification Report (includes Precision, Recall, F1-score)
    print("\nClassification Report:")
    print(classification_report(y_true_binary, binary_predictions, target_names=['Benign', 'Malware']))

    # Plot and Save Confusion Matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Benign', 'Malware'], yticklabels=['Benign', 'Malware'])
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')

    # Save the plot
    confusion_matrix_filepath = os.path.join(PLOTS_SAVE_FOLDER, 'confusion_matrix_plot.png')
    plt.tight_layout()
    plt.savefig(confusion_matrix_filepath)
    print(f"Saved confusion matrix plot to {confusion_matrix_filepath}")
    plt.close() # Close the plot figure

else:
    print("\n'label' column not found in the test data. Cannot calculate evaluation metrics or plot confusion matrix.")

## ROC Curve and AUC Score

Generate and plot the Receiver Operating Characteristic (ROC) curve and calculate the Area Under the Curve (AUC) score. This helps evaluate the model's ability to distinguish between the two classes.

In [ ]:
# --- Step 9: Plot ROC Curve and calculate AUC (if true labels available) ---

from sklearn.metrics import roc_curve, auc, RocCurveDisplay # Import necessary metrics

if 'label' in combined_test_df_binary.columns:
    print("\n" + "="*50)
    print("ROC CURVE AND AUC SCORE (if true labels available)")
    print("="*50)

    y_true_binary = combined_test_df_binary['label']

    # Assuming the binary model predicts probabilities (most classifiers do)
    # If your model has a decision_function instead of predict_proba, you'll need to adjust
    if hasattr(loaded_binary_model, 'predict_proba'):
        y_scores_binary = loaded_binary_model.predict_proba(X_test_scaled_binary)[:, 1] # Probability of the positive class (Malware)

        # Calculate ROC curve
        fpr, tpr, thresholds = roc_curve(y_true_binary, y_scores_binary)

        # Calculate AUC
        roc_auc = auc(fpr, tpr)
        print(f"\nAUC (Area Under the Curve): {roc_auc:.4f}")

        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

    elif hasattr(loaded_binary_model, 'decision_function'):
         print("\nModel has decision_function. ROC curve plotting from decision_function is not implemented here.")
         # You would need to calculate fpr, tpr, and thresholds from decision_function if needed.
         # y_scores_binary = loaded_binary_model.decision_function(X_test_scaled_binary)
         # fpr, tpr, thresholds = roc_curve(y_true_binary, y_scores_binary)
         # roc_auc = auc(fpr, tpr)
         # print(f"\nAUC (Area Under the Curve): {roc_auc:.4f}")
         # ... plotting code ...

    else:
        print("\nWarning: The loaded binary model does not have 'predict_proba' or 'decision_function'. Cannot plot ROC curve.")

else:
    print("\n'label' column not found in the test data. Cannot calculate AUC or plot ROC curve.")

# **Log CPU and memory usage before and after prediction,**

 **Store these logs in the "model_resource_usage_logs" folder in Google Drive, and visualize the current resource usage compared to historical usage from the logs.**


## Install necessary libraries

### Subtask:
Install libraries for monitoring system resources if needed.


**Reasoning**:
Check if psutil is installed and install it if necessary.



In [ ]:
import psutil

## Modify prediction code

### Subtask:
Add code to capture resource usage metrics (CPU, memory) before and after the model prediction step.


**Reasoning**:
Add code to capture resource usage metrics (CPU, memory) before and after the model prediction step using psutil.



In [ ]:
# --- Step 7: Make binary predictions ---

# Capture resource usage before prediction
cpu_percent_before = psutil.cpu_percent(interval=1) # Measure CPU usage over 1 second
memory_before = psutil.virtual_memory().used

# Make predictions on the scaled test data
if 'loaded_binary_model' in locals() and loaded_binary_model is not None:
    binary_predictions = loaded_binary_model.predict(X_test_scaled_binary)
    print("\nBinary predictions made on the test data.")

    # Capture resource usage after prediction
    cpu_percent_after = psutil.cpu_percent(interval=1) # Measure CPU usage over 1 second
    memory_after = psutil.virtual_memory().used

    # Calculate memory consumed during prediction
    memory_consumed = memory_after - memory_before

    print(f"\nCPU Usage Before Prediction: {cpu_percent_before}%")
    print(f"Memory Usage Before Prediction: {memory_before} bytes")
    print(f"CPU Usage After Prediction: {cpu_percent_after}%")
    print(f"Memory Usage After Prediction: {memory_after} bytes")
    print(f"Memory Consumed During Prediction: {memory_consumed} bytes")


    # Display the predictions (0: Benign, 1: Malware)
    print("\nSample Binary Predictions (0: Benign, 1: Malware):")
    print(binary_predictions[:20]) # Display first 20 predictions

    # Count the number of benign and malware predictions
    predicted_binary_counts = pd.Series(binary_predictions).value_counts().sort_index()
    print("\nPredicted Binary Counts (0: Benign, 1: Malware):")
    print(predicted_binary_counts)

else:
    raise Exception("Error: Binary model not loaded. Cannot make predictions.")

## Log resource usage

### Subtask:
Store the captured resource usage metrics in a structured format (e.g., CSV or a simple text file) within the specified Google Drive folder (`model_resource_usage_logs`). Include a timestamp or run identifier.


**Reasoning**:
Store the captured resource usage metrics in a structured format within the specified Google Drive folder.



In [ ]:
# --- Step 8: Log resource usage ---
from datetime import datetime
import os
import pandas as pd # Import pandas here

# Define the directory to save logs. This should be an absolute or relative path on your Raspberry Pi.
LOGS_DIR = '/home/pi/Desktop/iot/model_resource_usage_logs'
os.makedirs(LOGS_DIR, exist_ok=True) # Create the directory if it doesn't exist

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
LOG_FILE_PATH = os.path.join(LOGS_DIR, f'binary_prediction_resource_usage_{timestamp}.csv')

# Data to be logged
log_data = {
    'timestamp': timestamp,
    'cpu_percent_before': cpu_percent_before,
    'memory_before': memory_before,
    'cpu_percent_after': cpu_percent_after,
    'memory_after': memory_after,
    'memory_consumed': memory_consumed
}

# Create a DataFrame for easier handling
log_df = pd.DataFrame([log_data])

# Check if file exists to write header
if not os.path.exists(LOG_FILE_PATH):
    log_df.to_csv(LOG_FILE_PATH, index=False, mode='w')
else:
    log_df.to_csv(LOG_FILE_PATH, index=False, mode='a', header=False)

print(f"\nResource usage logged to: {LOG_FILE_PATH}")

## Read historical logs

### Subtask:
Load the historical resource usage data from the log files in the Google Drive folder.


**Reasoning**:
Load the historical resource usage data from the log files in the Google Drive folder.



In [ ]:
# --- Step 9: Load historical resource usage data ---
import os
import pandas as pd

# Define the directory where logs are saved on your Raspberry Pi.
LOGS_DIR = '/home/pi/Desktop/iot/model_resource_usage_logs'

print(f"\nLoading historical resource usage data from: {LOGS_DIR}")

all_log_files = []
for dirpath, _, filenames in os.walk(LOGS_DIR):
    for f in filenames:
        if f.endswith('.csv'):
            all_log_files.append(os.path.join(dirpath, f))

if not all_log_files:
    print(f"No historical log files found in {LOGS_DIR}.")
    historical_resource_usage_df = pd.DataFrame() # Create an empty DataFrame if no files found
else:
    all_log_dfs = []
    for file_path in all_log_files:
        try:
            log_df = pd.read_csv(file_path)
            all_log_dfs.append(log_df)
            # print(f"  Successfully loaded {file_path}") # Uncomment for detailed progress
        except Exception as e:
            print(f"  Error loading {file_path}: {e}")

    if all_log_dfs:
        historical_resource_usage_df = pd.concat(all_log_dfs, ignore_index=True)
        print("\nHistorical resource usage data combining complete.")
        print(f"Shape of historical resource usage data: {historical_resource_usage_df.shape}")
        # Display the first few rows and info to understand the structure
        print("\nFirst 5 rows of historical resource usage data:")
        print(historical_resource_usage_df.head())
    else:
        print("\nNo historical log files were successfully loaded.")
        historical_resource_usage_df = pd.DataFrame() # Create an empty DataFrame if no files successfully loaded

## Visualize resource usage trends

### Subtask:
Create plots (e.g., line plots or bar plots) to visualize the resource usage over different runs or timestamps, allowing for comparison of current usage with past runs.


**Reasoning**:
Convert the 'timestamp' column to datetime objects and then create line plots to visualize the historical CPU and memory usage after prediction.



In [ ]:
# --- Step 10: Visualize historical resource usage ---
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd # Import pandas here

if not historical_resource_usage_df.empty:
    print("\n" + "="*50)
    print("VISUALIZING HISTORICAL RESOURCE USAGE")
    print("="*50)

    # Convert 'timestamp' column to datetime objects
    historical_resource_usage_df['timestamp'] = pd.to_datetime(historical_resource_usage_df['timestamp'], format='%Y%m%d_%H%M%S')

    # Sort by timestamp to ensure correct plotting order
    historical_resource_usage_df = historical_resource_usage_df.sort_values('timestamp')

    # Create a line plot for CPU usage after prediction
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=historical_resource_usage_df, x='timestamp', y='cpu_percent_after', marker='o')
    plt.title('CPU Usage After Prediction Over Time')
    plt.xlabel('Timestamp')
    plt.ylabel('CPU Usage (%)')
    plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
    plt.tight_layout()
    plt.show()

    # Create a line plot for memory consumed during prediction
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=historical_resource_usage_df, x='timestamp', y='memory_consumed', marker='o', color='orange')
    plt.title('Memory Consumed During Prediction Over Time')
    plt.xlabel('Timestamp')
    plt.ylabel('Memory Consumed (bytes)')
    plt.xticks(rotation=45, ha='right') # Rotate x-axis labels for better readability
    plt.tight_layout()
    plt.show()

else:
    print("\nNo historical resource usage data available for visualization.")


# --- Step 11: Plot ROC Curve and calculate AUC (original code) ---

from sklearn.metrics import roc_curve, auc, RocCurveDisplay # Import necessary metrics

if 'label' in combined_test_df_binary.columns:
    print("\n" + "="*50)
    print("ROC CURVE AND AUC SCORE (if true labels available)")
    print("="*50)

    y_true_binary = combined_test_df_binary['label']

    # Assuming the binary model predicts probabilities (most classifiers does)
    # If your model has a decision_function instead of predict_proba, you'll need to adjust
    if hasattr(loaded_binary_model, 'predict_proba'):
        y_scores_binary = loaded_binary_model.predict_proba(X_test_scaled_binary)[:, 1] # Probability of the positive class (Malware)

        # Calculate ROC curve
        fpr, tpr, thresholds = roc_curve(y_true_binary, y_scores_binary)

        # Calculate AUC
        roc_auc = auc(fpr, tpr)
        print(f"\nAUC (Area Under the Curve): {roc_auc:.4f}")

        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

    elif hasattr(loaded_binary_model, 'decision_function'):
         print("\nModel has decision_function. ROC curve plotting from decision_function is not implemented here.")
         # You would need to calculate fpr, tpr, and thresholds from decision_function if needed.
         # y_scores_binary = loaded_binary_model.decision_function(X_test_scaled_binary)
         # fpr, tpr, thresholds = roc_curve(y_true_binary, y_scores_binary)
         # roc_auc = auc(fpr, tpr)
         # print(f"\nAUC (Area Under the Curve): {roc_auc:.4f}")
         # ... plotting code ...

    else:
        print("\nWarning: The loaded binary model does not have 'predict_proba' or 'decision_function'. Cannot plot ROC curve.")

else:
    print("\n'label' column not found in the test data. Cannot calculate AUC or plot ROC curve.")

## Summarize current usage

### Subtask:
Display the resource usage metrics for the current prediction run.


**Reasoning**:
Access and print the resource usage metrics from the `log_data` dictionary for the current prediction run, including appropriate labels and units.



In [ ]:
# --- Step 11: Display current resource usage ---

print("\n" + "="*50)
print("CURRENT RESOURCE USAGE")
print("="*50)

# Access the log_data dictionary (created in the logging step)
if 'log_data' in locals():
    print(f"Timestamp: {log_data.get('timestamp', 'N/A')}")
    print(f"CPU Usage Before Prediction: {log_data.get('cpu_percent_before', 'N/A')}%")
    print(f"Memory Usage Before Prediction: {log_data.get('memory_before', 'N/A')} bytes")
    print(f"CPU Usage After Prediction: {log_data.get('cpu_percent_after', 'N/A')}%")
    print(f"Memory Usage After Prediction: {log_data.get('memory_after', 'N/A')} bytes")
    print(f"Memory Consumed During Prediction: {log_data.get('memory_consumed', 'N/A')} bytes")
else:
    print("Resource usage data (log_data) is not available.")

# --- Step 12: Plot ROC Curve and calculate AUC (original code, remains unchanged) ---

from sklearn.metrics import roc_curve, auc # Import necessary metrics
import matplotlib.pyplot as plt # Import matplotlib here

if 'label' in combined_test_df_binary.columns:
    print("\n" + "="*50)
    print("ROC CURVE AND AUC SCORE (if true labels available)")
    print("="*50)

    y_true_binary = combined_test_df_binary['label']

    # Assuming the binary model predicts probabilities (most classifiers does)
    # If your model has a decision_function instead of predict_proba, you'll need to adjust
    if hasattr(loaded_binary_model, 'predict_proba'):
        y_scores_binary = loaded_binary_model.predict_proba(X_test_scaled_binary)[:, 1] # Probability of the positive class (Malware)

        # Calculate ROC curve
        fpr, tpr, thresholds = roc_curve(y_true_binary, y_scores_binary)

        # Calculate AUC
        roc_auc = auc(fpr, tpr)
        print(f"\nAUC (Area Under the Curve): {roc_auc:.4f}")

        # Plot ROC curve
        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

    elif hasattr(loaded_binary_model, 'decision_function'):
         print("\nModel has decision_function. ROC curve plotting from decision_function is not implemented here.")
         # You would need to calculate fpr, tpr, and thresholds from decision_function if needed.
         # y_scores_binary = loaded_binary_model.decision_function(X_test_scaled_binary)
         # fpr, tpr, thresholds = roc_curve(y_true_binary, y_scores_binary)
         # roc_auc = auc(fpr, tpr)
         # print(f"\nAUC (Area Under the Curve): {roc_auc:.4f}")
         # ... plotting code ...

    else:
        print("\nWarning: The loaded binary model does not have 'predict_proba' or 'decision_function'. Cannot plot ROC curve.")

else:
    print("\n'label' column not found in the test data. Cannot calculate AUC or plot ROC curve.")

In [ ]:
# # --- Isolated SVM Algorithm Code ---
# # This cell contains code for training the SVM model and was likely used in a Colab environment.
# # It is commented out as it is not needed for the prediction script intended for Raspberry Pi.

# # --- Step 1: Mount your Google Drive ---
# # This will prompt you for authorization. Follow the link to get your code.
# # from google.colab import drive
# # drive.mount('/content/drive')
# # print("✅ Google Drive mounted successfully.")

# # --- Step 2: Import all necessary libraries ---
# # import pandas as pd
# # import numpy as np
# # import matplotlib.pyplot as plt
# # import seaborn as sns
# # from sklearn.model_selection import train_test_split
# # from sklearn.preprocessing import StandardScaler
# # from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve, auc, roc_auc_score
# # from imblearn.over_sampling import SMOTE
# # from sklearn.svm import SVC
# # import joblib
# # import os
# # import time
# # import warnings
# # warnings.filterwarnings('ignore')


# # --- Step 3: Set the path to your dataset and model save directory ---
# # DATASET_ROOT_PATH = '/content/drive/MyDrive/Colab Notebooks/dataset'
# # MODEL_SAVE_DIR = '/content/drive/MyDrive/Colab Notebooks/ML_Models/2_RF_Early_Version'
# # os.makedirs(MODEL_SAVE_DIR, exist_ok=True) # Ensure the directory exists


# # --- Step 4: Load and combine data ---
# # List available CSV files by walking through the directory and its subdirectories
# # data_files = []
# # for dirpath, _, filenames in os.walk(DATASET_ROOT_PATH):
# #     for f in filenames:
# #         if f.endswith('.csv'):
# #             data_files.append(os.path.join(dirpath, f))

# # print(f"\nFound {len(data_files)} data files")
# # print("Sample files:", data_files[:5])


# # Load N-BaIoT dataset with labels
# # def load_iot_malware_data(sample_size=10000):
# #     """
# #     Load N-BaIoT dataset with labels
# #     sample_size: samples per device/attack type for memory efficiency
# #     """
# #     all_data = []

# #     # Define device types and attack types
# #     devices = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell',
# #                'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera',
# #                'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam',
# #                'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']

# #     attacks = ['gafgyt', 'mirai']  # Focus on Mirai and Gafgyt (BASHLITE)

# #     # Load benign data
# #     print("\nLoading benign samples...")
# #     benign_count = 0
# #     for device in devices:
# #         for file in data_files:
# #             if device in file and 'benign' in file.lower():
# #                 try:
# #                     df = pd.read_csv(file, nrows=sample_size)
# #                     df['label'] = 0  # Benign
# #                     df['device'] = device
# #                     df['attack_type'] = 'benign'
# #                     all_data.append(df)
# #                     benign_count += len(df)
# #                 except Exception as e:
# #                     continue

# #     # Load malware data
# #     print("\nLoading malware samples...")
# #     malware_count = 0
# #     for device in devices:
# #         for attack in attacks:
# #             for file in data_files:
# #                 if device in file and attack in file.lower():
# #                     try:
# #                         df = pd.read_csv(file, nrows=sample_size)
# #                         df['label'] = 1  # Malware
# #                         df['device'] = device
# #                         df['attack_type'] = attack
# #                         all_data.append(df)
# #                         malware_count += len(df)
# #                     except Exception as e:
# #                         continue

# #     # Combine all data
# #     if all_data:
# #         combined_df = pd.concat(all_data, ignore_index=True)
# #         print(f"\nTotal samples loaded: {len(combined_df)}")
# #         print(f"Benign: {benign_count}, Malware: {malware_count}")
# #         print(f"Class balance: {malware_count/(benign_count+malware_count)*100:.1f}% malware")
# #         return combined_df
# #     else:
# #         raise Exception("No data loaded. Please check file paths and contents.")

# # # Load the data using the function defined previously
# # print("Loading N-BaIoT dataset...")
# # df = load_iot_malware_data(sample_size=5000)

# # print("\nData loaded successfully.")


# # --- Step 5: Create a smaller subset of the data for SVM training ---
# # print("\n" + "="*50)
# # print("CREATING SUBSET FOR SVM")
# # print("="*50)

# # # Define the desired sample size per device/attack type for the SVM subset
# # SVM_SAMPLE_SIZE = 1000

# # # Create an empty list to store the sampled dataframes
# # svm_subset_data = []

# # devices = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell',
# #            'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera',
# #            'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam',
# #            'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']

# # attacks = ['gafgyt', 'mirai', 'benign'] # Include benign for subsetting

# # for device in devices:
# #     for attack in attacks:
# #         # Filter the dataframe for the current device and attack type
# #         subset_df = df[(df['device'] == device) & (df['attack_type'] == attack)]

# #         # Sample from this subset
# #         if len(subset_df) > SVM_SAMPLE_SIZE:
# #             sampled_subset = subset_df.sample(n=SVM_SAMPLE_SIZE, random_state=42)
# #         else:
# #             sampled_subset = subset_df # Take all if less than sample size

# #         svm_subset_data.append(sampled_subset)

# # # Concatenate the sampled dataframes
# # df_svm = pd.concat(svm_subset_data, ignore_index=True)

# # print(f"\nTotal samples in SVM subset: {len(df_svm)}")
# # print(f"SVM subset class distribution:\n{df_svm['label'].value_counts()}")


# # --- Step 6: Separate features and labels, then split data ---
# # Use the same feature columns as the Undersampled RF for fair comparison
# # feature_columns = [col for col in df.columns if col not in ['label', 'device', 'attack_type']]
# # feature_columns_svm = feature_columns # Use the full feature columns

# # X_svm = df_svm[feature_columns_svm]
# # y_svm = df_svm['label']

# # print(f"\nSVM subset feature shape: {X_svm.shape}")

# # # Split the SVM subset data
# # X_train_svm, X_test_svm, y_train_svm, y_test_svm = train_test_split(
# #     X_svm, y_svm, test_size=0.2, random_state=42, stratify=y_svm
# # )

# # print(f"\nSVM Train set: {X_train_svm.shape}, SVM Test set: {X_test_svm.shape}")
# # print(f"SVM Train class distribution: {y_train_svm.value_counts().to_dict()}")
# # print(f"SVM Test class distribution: {y_test_svm.value_counts().to_dict()}")


# # --- Step 7: Handle class imbalance with SMOTE and Scale features for SVM ---
# # print("\n" + "="*50)
# # print("HANDLING CLASS IMBALANCE WITH SMOTE & SCALING FOR SVM SUBSET")
# # print("="*50)

# # # Apply SMOTE to the SVM training subset
# # smote_svm = SMOTE(random_state=42)
# # X_train_svm_balanced, y_train_svm_balanced = smote_svm.fit_resample(X_train_svm, y_train_svm)

# # print(f"Original SVM training subset: {X_train_svm.shape}")
# # print(f"Balanced SVM training subset: {X_train_svm_balanced.shape}")
# # print(f"New SVM class distribution: {pd.Series(y_train_svm_balanced).value_counts().to_dict()}")

# # # Scale features for SVM using a new scaler fitted on the SVM training data
# # scaler_svm = StandardScaler()
# # X_train_svm_scaled = scaler_svm.fit_transform(X_train_svm_balanced)
# # X_test_svm_scaled = scaler_svm.transform(X_test_svm) # Scale the SVM test subset

# # # Save the SVM scaler for later use
# # scaler_svm_filename = os.path.join(MODEL_SAVE_DIR, "iot_malware_svm_subset_scaler.pkl")
# # joblib.dump(scaler_svm, scaler_svm_filename)
# # print(f"Saved SVM scaler as {scaler_svm_filename}")


# # --- Step 8: Train SVM ---
# # print("\n" + "="*50)
# # print("TRAINING SVM CLASSIFIER")
# # print("="*50)

# # # Initialize and train SVM (using RBF kernel)
# # svm_model = SVC(
# #     kernel='rbf',
# #     C=1.0,
# #     gamma='scale',
# #     random_state=42
# # )

# # # Measure training time
# # start_time = time.time()
# # # Train SVM on the scaled and SMOTE-balanced SVM training subset
# # svm_model.fit(X_train_svm_scaled, y_train_svm_balanced)
# # svm_train_time = time.time() - start_time

# # print(f"SVM Training time: {svm_train_time:.2f} seconds")

# # # Save the trained SVM model
# # svm_model_filename = os.path.join(MODEL_SAVE_DIR, "iot_malware_svm_subset_model.pkl")
# # joblib.dump(svm_model, svm_model_filename)
# # print(f"Saved SVM model as {svm_model_filename}")


# # --- Step 9: Evaluate SVM ---
# # print("\n" + "="*50)
# # print("EVALUATING SVM PERFORMANCE")
# # print("="*50)

# # # Make predictions on the scaled SVM test subset
# # svm_pred = svm_model.predict(X_test_svm_scaled)

# # # Evaluate
# # svm_accuracy = accuracy_score(y_test_svm, svm_pred)
# # svm_f1 = f1_score(y_test_svm, svm_pred, average='weighted')

# # print(f"SVM Accuracy: {svm_accuracy:.4f}")
# # print(f"SVM F1-score: {svm_f1:.4f}")
# # print("\nSVM Classification Report:")
# # print(classification_report(y_test_svm, svm_pred, target_names=['Benign', 'Malware']))

# # # SVM Confusion matrix (using the SVM test subset)
# # cm_svm = confusion_matrix(y_test_svm, svm_pred)

# # plt.figure(figsize=(6, 5))
# # sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Blues',
# #             xticklabels=['Benign', 'Malware'], yticklabels=['Benign', 'Malware'])
# # plt.title('SVM Confusion Matrix (on Subset)')
# # plt.xlabel('Predicted')
# # plt.ylabel('Actual')
# # plt.show()

# # # Calculate FPR for the SVM Model
# # print("\n" + "="*50)
# # print("FPR CALCULATION (SVM)")
# # print("="*50)

# # # Calculate False Positive Rate (FPR)
# # tn_svm, fp_svm, fn_svm, tp_svm = cm_svm.ravel()
# # fpr_svm = fp_svm / (fp_svm + tn_svm)
# # print(f"\nFalse Positive Rate (FPR) (SVM): {fpr_svm:.4f}")

# # # ROC AUC Score (SVM) - Note: SVC with default settings does not have predict_proba
# # # You would need probability=True in SVC initialization for this.
# # # try:
# # #     svm_pred_proba = svm_model.predict_proba(X_test_svm_scaled)[:, 1] # Get probability of the positive class (Malware=1)
# # #     roc_auc_svm = roc_auc_score(y_test_svm, svm_pred_proba)
# # #     print(f"ROC AUC Score (SVM): {roc_auc_svm:.4f}")

# # #     # Optionally, plot the ROC curve
# # #     fpr_curve_svm, tpr_curve_svm, thresholds_svm = roc_curve(y_test_svm, svm_pred_proba)
# # #     plt.figure(figsize=(8, 6))
# # #     plt.plot(fpr_curve_svm, tpr_curve_svm, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc_svm:.4f})')
# # #     plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
# # #     plt.xlim([0.0, 1.0])
# # #     plt.ylim([0.0, 1.05])
# # #     plt.xlabel('False Positive Rate')
# # #     plt.ylabel('True Positive Rate')
# # #     plt.title('Receiver Operating Characteristic (ROC) Curve (SVM)')
# # #     plt.legend(loc="lower right")
# # #     plt.show()

# # # except AttributeError:
# # #     print("\nWarning: The trained SVM model does not have a 'predict_proba' method.")
# # #     print("Cannot calculate ROC AUC score for SVM without prediction probabilities.")
# # # except Exception as e:
# # #     print(f"\nAn error occurred during SVM ROC AUC calculation: {e}")

In [ ]:
# # import pandas as pd
# # import numpy as np
# # import joblib
# # import glob
# # import os
# # import time
# # from datetime import datetime
# # from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# # import matplotlib.pyplot as plt
# # import seaborn as sns
# # import resource # This module might provide basic resource usage info on Unix-like systems

# # # --- Configuration ---
# # # !!! UPDATE THESE PATHS to match your Raspberry Pi file system !!!
# # # Assuming your test data CSVs are in a folder named 'test_data' in your home directory or similar
# # TEST_DATA_FOLDER = '/home/pi/Desktop/iot/test_data/' # Example path - UPDATED!
# # # Update these paths to your SVM model and scaler files
# # MODEL_PATH = '/home/pi/Desktop/iot/model/iot_malware_svm_subset_model.pkl' # Example path - UPDATED!
# # SCALER_PATH = '/home/pi/Desktop/iot/model/iot_malware_svm_subset_scaler.pkl' # Example path - UPDATED!
# # # Folder to save plots
# # PLOTS_SAVE_FOLDER = '/home/pi/Desktop/iot/plots/' # New folder for saving plots - UPDATED!


# # # IMPORTANT: While we are loading all columns, the scaler and model expect
# # # features in a specific order and number. Ensure your test data CSVs
# # # have the same columns in the same order as your training data.
# # # The FEATURE_COLUMNS list is now primarily for reference/understanding,
# # # but the data loading won't strictly enforce it line by line.
# # FEATURE_COLUMNS = ['HH_jit_L0_1_mean', 'HH_jit_L0_01_mean', 'HpHp_L0_01_radius',
# #                    'H_L0_01_weight', 'MI_dir_L0_1_weight'] # Keep as reference, update if known

# # # Number of random samples to test
# # NUM_TEST_SAMPLES = 5000 # New configuration for sample size


# # # --- Data Loading Function ---
# # def load_test_data_from_folder(folder_path): # Removed feature_columns parameter
# #     """
# #     Loads all CSV files from the specified folder and concatenates them.
# #     Loads ALL columns from the CSV files.
# #     Adds a dummy 'true_label' column for simulation purposes.
# #     In a real scenario, you need a way to get actual true labels.
# #     """
# #     all_files = glob.glob(os.path.join(folder_path, "*.csv"))
# #     if not all_files:
# #         print(f"No CSV files found in {folder_path}")
# #         return pd.DataFrame(), pd.Series() # Return empty DataFrame and Series

# #     list_dfs = []
# #     list_true_labels = [] # To store simulated true labels

# #     print(f"Loading test data from {folder_path}...")
# #     for filename in all_files:
# #         try:
# #             # Load the CSV file - this will load ALL columns
# #             df = pd.read_csv(filename)

# #             list_dfs.append(df)

# #             # --- Simulate True Labels ---
# #             # !!! IMPORTANT: Replace this with your actual logic to get true labels !!!
# #             # Example: If file name contains 'malware', assume all samples are malware (1)
# #             # If file name contains 'normal', assume all samples are normal (0)
# #             # Otherwise, default to 0 (normal)
# #             true_label = 0
# #             if 'malware' in filename.lower():
# #                 true_label = 1
# #             elif 'normal' in filename.lower():
# #                 true_label = 0
# #             else:
# #                  # Default or derive from another source
# #                  pass # Keep default 0, or implement your logic


# #             list_true_labels.extend([true_label] * len(df)) # Add labels for all rows in the file


# #             print(f"Loaded {filename} ({len(df)} samples), simulated label: {true_label}")

# #         except Exception as e:
# #             print(f"Error loading {filename}: {e}")

# #     if list_dfs:
# #         test_df = pd.concat(list_dfs, ignore_index=True)
# #         true_labels_series = pd.Series(list_true_labels, name='true_label')

# #         # --- IMPORTANT CHECK ---
# #         # After loading all data, it's crucial that the resulting DataFrame
# #         # has the EXACT same columns in the EXACT same order as the data
# #         # the scaler and model were trained on. If not, the scaler.transform()
# #         # call below will likely fail or produce incorrect results.
# #         # We cannot automatically verify this here without the original training data.
# #         # You might need to manually inspect the columns of test_df and compare
# #         # them to your training data columns.

# #         print(f"Loaded total {len(test_df)} samples for testing.")
# #         print(f"Loaded columns: {list(test_df.columns)}") # Print loaded columns for verification

# #         return test_df, true_labels_series
# #     else:
# #         return pd.DataFrame(), pd.Series()


# # # --- Main Script Execution ---
# # if __name__ == "__main__":
# #     print("Starting test script...")

# #     # Create the plots save folder if it doesn't exist
# #     os.makedirs(PLOTS_SAVE_FOLDER, exist_ok=True)
# #     print(f"Ensured plots save folder exists: {PLOTS_SAVE_FOLDER}")


# #     # --- 1. Load Model and Scaler ---
# #     start_time_load = time.time()
# #     try:
# #         model = joblib.load(MODEL_PATH)
# #         scaler = joblib.load(SCALER_PATH)
# #         print("✅ Successfully loaded model and scaler.")
# #     except FileNotFoundError as e:
# #         print(f"❌ Error: Model or scaler file not found: {e}")
# #         print("Please update the MODEL_PATH and SCALER_PATH variables to the correct locations on your Pi.")
# #         exit() # Stop execution if files are not found
# #     except Exception as e:
# #         print(f"❌ Error loading model or scaler: {e}")
# #         exit()


# #     end_time_load = time.time()
# #     load_time = end_time_load - start_time_load
# #     print(f"⏱️ Model and Scaler loading time: {load_time:.4f} seconds")

# #     # --- 2. Load Test Data ---
# #     start_time_data = time.time()
# #     # Load all data first
# #     all_test_data_df, all_true_labels = load_test_data_from_folder(TEST_DATA_FOLDER)
# #     end_time_data = time.time()
# #     data_load_time = end_time_data - start_time_data
# #     print(f"⏱️ Full data loading time: {data_load_time:.4f} seconds")


# #     if all_test_data_df.empty:
# #         print("❌ No test data loaded. Cannot proceed with predictions.")
# #         exit() # Stop execution if no data


# #     # --- 3. Select a random subset ---
# #     print(f"\nSelecting a random subset of {NUM_TEST_SAMPLES} samples...")
# #     if len(all_test_data_df) < NUM_TEST_SAMPLES:
# #         print(f"Warning: Only {len(all_test_data_df)} samples available, testing on all loaded data.")
# #         test_data_df = all_test_data_df.copy()
# #         true_labels = all_true_labels.copy()
# #     else:
# #         # Randomly sample the DataFrame and corresponding labels
# #         random_indices = np.random.choice(all_test_data_df.index, size=NUM_TEST_SAMPLES, replace=False)
# #         test_data_df = all_test_data_df.loc[random_indices].reset_index(drop=True)
# #         true_labels = all_true_labels.loc[random_indices].reset_index(drop=True)

# #     print(f"Testing on {len(test_data_df)} samples.")


# #     # Check if simulated true labels match data length
# #     if len(test_data_df) != len(true_labels):
# #          print("Warning: Number of test data samples and simulated true labels do not match after sampling!")
# #          # This should not happen with the current sampling logic if all_true_labels matched all_test_data_df
# #          # but keep the check for safety.
# #          min_len = min(len(test_data_df), len(true_labels))
# #          test_data_df = test_data_df.head(min_len)
# #          true_labels = true_labels.head(min_len)
# #          print(f"Adjusted sampled data and labels to {min_len} samples for comparison.")


# #     # --- 4. Make Predictions ---
# #     start_time_predict = time.time()

# #     try:
# #         # Scale the test data subset
# #         print("Scaling test data subset...")
# #         # This call assumes test_data_df has the correct columns in the correct order
# #         # which is consistent with the full data loaded earlier
# #         test_data_scaled = scaler.transform(test_data_df)

# #         # Make predictions
# #         print("Making predictions...")
# #         predictions = model.predict(test_data_scaled)

# #         print("✅ Predictions completed.")
# #     except Exception as e:
# #         print(f"❌ Error during prediction (likely column mismatch or scaling issue): {e}")
# #         print("Please verify the columns in your test data CSVs match the data used to train the scaler/model.")
# #         exit()


# #     end_time_predict = time.time()
# #     prediction_time = end_time_predict - start_time_predict
# #     print(f"⏱️ Prediction time for {len(test_data_df)} samples: {prediction_time:.4f} seconds")
# #     if len(test_data_df) > 0:
# #         print(f"Average prediction time per sample: {(prediction_time / len(test_data_df)):.6f} seconds")


# #     # --- 5. Evaluate Performance (requires true labels) ---
# #     print("\n--- Performance Evaluation ---")
# #     if not true_labels.empty and len(true_labels) == len(predictions):
# #         try:
# #             # Ensure true_labels and predictions have the same index for correct comparison
# #             true_labels = true_labels.reset_index(drop=True)
# #             predictions_series = pd.Series(predictions).reset_index(drop=True)


# #             accuracy = accuracy_score(true_labels, predictions)
# #             precision = precision_score(true_labels, predictions, zero_division=0) # Handle cases with no positive predictions
# #             recall = recall_score(true_labels, predictions, zero_division=0)     # Handle cases with no positive true labels
# #             f1 = f1_score(true_labels, predictions, zero_division=0)             # Handle cases with no positive true/predicted

# #             print(f"Accuracy: {accuracy:.4f}")
# #             print(f"Precision (Malware): {precision:.4f}")
# #             print(f"Recall (Malware): {recall:.4f}")
# #             f1 = f1_score(true_labels, predictions, zero_division=0)             # Handle cases with no positive true/predicted
# #             print(f"F1-Score (Malware): {f1:.4f}")
# #             print("\nConfusion Matrix:")
# #             conf_matrix = confusion_matrix(true_labels, predictions)
# #             print(conf_matrix)

# #             # Optional: Print classification report for more detail
# #             # try:
# #             #      print("\nClassification Report:")
# #             #      from sklearn.metrics import classification_report
# #             #      print(classification_report(true_labels, predictions, zero_division=0))
# #             # except ImportError:
# #             #     print("sklearn.metrics.classification_report not available.")
# #             # except Exception as e:
# #             #      print(f"Error generating classification report: {e}")


# #         except Exception as e:
# #             print(f"❌ Error calculating performance metrics: {e}")
# #             print("Performance evaluation skipped.")
# #             # Set metrics to None or default for plotting
# #             accuracy, precision, recall, f1, conf_matrix = None, None, None, None, None
# #     else:
# #         print("⚠️ Cannot evaluate performance: True labels not available or mismatch with predictions.")
# #         accuracy, precision, recall, f1, conf_matrix = None, None, None, None, None


# #     # --- 6. Report Basic Resource Usage (Placeholder) ---
# #     print("\n--- Resource Usage (Basic) ---")
# #     try:
# #         # Get resource usage (might not be available or detailed on all systems/environments)
# #         # This gives maximum resident set size (memory usage)
# #         mem_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
# #         # On Linux, ru_maxrss is typically in KB, but on macOS it's in bytes.
# #         # We'll assume KB for Raspberry Pi OS (Linux).
# #         print(f"Peak Memory Usage (Max RSS): {mem_usage} KB")

# #         # Getting CPU usage over time is more complex and usually requires monitoring tools
# #         print("CPU Usage: Monitoring requires external tools or specific libraries.")

# #     except NameError:
# #         print("❌ 'resource' module not found (usually available on Unix-like systems). Resource usage reporting skipped.")
# #     except Exception as e:
# #         print(f"❌ Could not get basic resource usage: {e}")
# #         print("Resource usage reporting skipped.")


# #     # --- 7. Plotting Results and Saving ---
# #     print("\n--- Generating and Saving Plots ---")

# #     if not true_labels.empty and len(true_labels) == len(predictions):
# #          try:
# #              # Create a DataFrame for plotting
# #              # Ensure indices align for plotting
# #              plot_df = pd.DataFrame({
# #                  'True Label': true_labels.reset_index(drop=True),
# #                  'Prediction': pd.Series(predictions).reset_index(drop=True),
# #                  'Sample Index': range(len(true_labels)) # Use a simple range as index for plotting
# #              })

# #              # Simple plot of True vs. Predicted labels over sample index
# #              plt.figure(figsize=(12, 6))
# #              # Use jitter for better visualization of overlapping points
# #              sns.stripplot(data=plot_df, x='Sample Index', y='True Label', label='True Label (0=Normal, 1=Malware)', alpha=0.6, jitter=0.2)
# #              sns.stripplot(data=plot_df, x='Sample Index', y='Prediction', label='Prediction (0=Normal, 1=Malware)', alpha=0.6, marker='x', color='red', jitter=0.2)

# #              plt.title('True Labels vs. Predictions Over Samples')
# #              plt.xlabel('Sample Index')
# #              plt.ylabel('Label (0=Normal, 1=Malware)')
# #              plt.yticks([0, 1])
# #              plt.legend()
# #              plt.grid(axis='y', linestyle='--')

# #              # Save the plot
# #              plot1_filename = os.path.join(PLOTS_SAVE_FOLDER, 'true_vs_predictions_plot.png')
# #              plt.savefig(plot1_filename)
# #              print(f"✅ Saved plot to {plot1_filename}")
# #              plt.close() # Close the plot figure to free memory


# #              # Plot Confusion Matrix (if available)
# #              if conf_matrix is not None:
# #                   plt.figure(figsize=(8, 6))
# #                   sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
# #                               xticklabels=['Predicted Normal', 'Predicted Malware'],
# #                               yticklabels=['True Normal', 'True Malware'])
# #                   plt.title('Confusion Matrix')
# #                   plt.xlabel('Predicted Label')
# #                   plt.ylabel('True Label')

# #                   # Save the plot
# #                   plot2_filename = os.path.join(PLOTS_SAVE_FOLDER, 'confusion_matrix_plot.png')
# #                   plt.savefig(plot2_filename)
# #                   print(f"✅ Saved plot to {plot2_filename}")
# #                   plt.close() # Close the plot figure


# #          except Exception as e:
# #              print(f"❌ Error generating or saving plots: {e}")
# #              print("Plotting skipped.")

# #     else:
# #         print("⚠️ Plotting requires both test data and corresponding true labels.")


# #     print("\nScript finished.")

In [ ]:
# # --- Raspberry Pi Stress Test and Accuracy Script ---

# import pandas as pd
# import numpy as np
# import joblib
# import glob
# import os
# import time
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# import resource # For basic resource usage on Unix-like systems
# import matplotlib.pyplot as plt # Import matplotlib for plotting
# import seaborn as sns # Import seaborn for enhanced plotting

# # --- Configuration ---
# # !!! UPDATE THESE PATHS to match your Raspberry Pi file system !!!
# TEST_DATA_FOLDER = '/home/pi/Desktop/iot/test_data/' # Example path - UPDATE THIS!
# MODEL_PATH = '/home/pi/Desktop/iot/model/iot_malware_svm_subset_model.pkl' # Example path - UPDATE THIS!
# SCALER_PATH = '/home/pi/Desktop/iot/model/iot_malware_svm_subset_scaler.pkl' # Example path - UPDATE THIS!
# PLOTS_SAVE_FOLDER = '/home/pi/Desktop/iot/test_output/plots/' # Define folder to save plots - NEW!


# # --- Data Loading Function ---
# def load_test_data_with_labels(folder_path):
#     """
#     Loads all CSV files from the specified folder, assuming they contain
#     both features and a 'label' column with true labels (0 for Benign, 1 for Malware).
#     Returns the features (X) and true labels (y).
#     """
#     all_files = glob.glob(os.path.join(folder_path, "*.csv"))
#     if not all_files:
#         print(f"No CSV files found in {folder_path}")
#         return pd.DataFrame(), pd.Series() # Return empty DataFrame and Series

#     list_dfs = []

#     print(f"Loading test data with true labels from {folder_path}...")
#     for filename in all_files:
#         try:
#             # Load the CSV file
#             df = pd.read_csv(filename)

#             # Ensure the 'label' column exists
#             if 'label' not in df.columns:
#                 print(f"Warning: 'label' column not found in {filename}. Skipping file.")
#                 continue

#             list_dfs.append(df)
#             print(f"Loaded {filename} ({len(df)} samples)")

#         except Exception as e:
#             print(f"Error loading {filename}: {e}")

#     if list_dfs:
#         combined_df = pd.concat(list_dfs, ignore_index=True)
#         print(f"\nLoaded total {len(combined_df)} samples for testing.")

#         # Separate features (X) and labels (y)
#         if 'label' in combined_df.columns:
#             y_true = combined_df['label']
#             X_test_raw = combined_df.drop(columns=['label'], errors='ignore') # Drop label, ignore if already dropped
#             print("Separated features and true labels.")
#             return X_test_raw, y_true
#         else:
#             print("Error: 'label' column not found in combined data after loading.")
#             return pd.DataFrame(), pd.Series()

#     else:
#         return pd.DataFrame(), pd.Series()


# # --- Main Script Execution ---
# if __name__ == "__main__":
#     print("Starting Raspberry Pi Stress Test and Accuracy Script...")

#     # Create the plots save folder if it doesn't exist
#     os.makedirs(PLOTS_SAVE_FOLDER, exist_ok=True)
#     print(f"Ensured plots save folder exists: {PLOTS_SAVE_FOLDER}")


#     # --- 1. Load Model and Scaler ---
#     start_time_load = time.time()
#     try:
#         model = joblib.load(MODEL_PATH)
#         scaler = joblib.load(SCALER_PATH)
#         print("✅ Successfully loaded model and scaler.")
#     except FileNotFoundError as e:
#         print(f"❌ Error: Model or scaler file not found: {e}")
#         print("Please update the MODEL_PATH and SCALER_PATH variables to the correct locations on your Pi.")
#         exit() # Stop execution if files are not found
#     except Exception as e:
#         print(f"❌ Error loading model or scaler: {e}")
#         exit()
#     end_time_load = time.time()
#     load_time = end_time_load - start_time_load
#     print(f"⏱️ Model and Scaler loading time: {load_time:.4f} seconds")


#     # --- 2. Load Test Data with True Labels ---
#     # This function assumes your test data CSVs contain a 'label' column (0 or 1)
#     start_time_data = time.time()
#     X_test_raw, y_true = load_test_data_with_labels(TEST_DATA_FOLDER)
#     end_time_data = time.time()
#     data_load_time = end_time_data - start_time_data
#     print(f"⏱️ Data loading time: {data_load_time:.4f} seconds")

#     if X_test_raw.empty or y_true.empty or len(X_test_raw) != len(y_true):
#         print("❌ Test data or true labels not loaded correctly. Cannot proceed with predictions and evaluation.")
#         print("Please ensure your test CSVs are in the specified folder and contain a 'label' column.")
#         exit() # Stop execution if data is not loaded correctly

#     print(f"Loaded {len(X_test_raw)} samples for testing.")

#     # --- 3. Prepare Data for Prediction (Align and Scale) ---
#     print("\nPreparing data for prediction (aligning and scaling)...")
#     try:
#         # Align columns with the scaler's expected features
#         scaler_feature_names = scaler.feature_names_in_

#         # Add missing columns to X_test_raw with default value (e.g., 0)
#         missing_cols_test = set(scaler_feature_names) - set(X_test_raw.columns)
#         for c in missing_cols_test:
#             X_test_raw[c] = 0

#         # Ensure the order of columns in X_test_raw is the same as in scaler_feature_names
#         X_test_aligned = X_test_raw[scaler_feature_names]

#         # Scale the aligned test data
#         X_test_scaled = scaler.transform(X_test_aligned)
#         print("✅ Test data scaled successfully.")

#     except Exception as e:
#         print(f"❌ Error during data preparation (alignment or scaling): {e}")
#         print("Please verify the columns in your test data CSVs match the data used to train the scaler/model.")
#         exit()


#     # --- 4. Make Predictions (Stress Test Component) ---
#     print("\nMaking predictions (stress test)...")
#     start_time_predict = time.time()

#     try:
#         # Make predictions on the scaled test data
#         predictions = model.predict(X_test_scaled)
#         print("✅ Predictions completed.")

#     except Exception as e:
#         print(f"❌ Error during prediction: {e}")
#         exit()

#     end_time_predict = time.time()
#     prediction_time = end_time_predict - start_time_predict
#     print(f"⏱️ Prediction time for {len(X_test_scaled)} samples: {prediction_time:.4f} seconds")
#     if len(X_test_scaled) > 0:
#         print(f"Average prediction time per sample: {(prediction_time / len(X_test_scaled)):.6f} seconds")

#     # --- 5. Evaluate Performance (Accuracy and other metrics) ---
#     print("\n--- Performance Evaluation ---")
#     try:
#         # Ensure true_labels and predictions have the same index for correct comparison
#         y_true = y_true.reset_index(drop=True)
#         predictions_series = pd.Series(predictions).reset_index(drop=True)

#         accuracy = accuracy_score(y_true, predictions)
#         precision = precision_score(y_true, predictions, zero_division=0)
#         recall = recall_score(y_true, predictions, zero_division=0)
#         f1 = f1_score(y_true, predictions, zero_division=0)
#         conf_matrix = confusion_matrix(y_true, predictions)

#         print(f"Accuracy: {accuracy:.4f}")
#         print(f"Precision (Malware): {precision:.4f}")
#         print(f"Recall (Malware): {recall:.4f}")
#         print(f"F1-Score (Malware): {f1:.4f}")
#         print("\nConfusion Matrix:")
#         print(conf_matrix)

#         # Optional: Print classification report
#         try:
#              from sklearn.metrics import classification_report
#              print("\nClassification Report:")
#              print(classification_report(y_true, predictions, target_names=['Benign', 'Malware'], zero_division=0))
#         except ImportError:
#             print("sklearn.metrics.classification_report not available.")
#         except Exception as e:
#              print(f"Error generating classification report: {e}")


#     except Exception as e:
#         print(f"❌ Error calculating performance metrics: {e}")
#         print("Performance evaluation skipped.")
#         conf_matrix = None # Ensure conf_matrix is None if calculation fails


#     # --- 6. Report Basic Resource Usage ---
#     print("\n--- Resource Usage (Basic) ---")
#     try:
#         # Get resource usage (might not be available or detailed on all systems/environments)
#         # This gives maximum resident set size (memory usage)
#         mem_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
#         # On Linux, ru_maxrss is typically in KB.
#         print(f"Peak Memory Usage (Max RSS): {mem_usage} KB")

#         # Getting CPU usage over time is more complex and usually requires monitoring tools
#         # For a simple snapshot, you could use psutil if installed:
#         # import psutil
#         # print(f"Current CPU Usage: {psutil.cpu_percent(interval=1)}%")
#         print("CPU Usage: For detailed monitoring, consider tools like 'htop' or 'atop' on your Pi.")

#     except NameError:
#         print("❌ 'resource' module not found (usually available on Unix-like systems). Resource usage reporting skipped.")
#     except Exception as e:
#         print(f"❌ Could not get basic resource usage: {e}")
#         print("Resource usage reporting skipped.")


#     # --- 7. Plotting Results and Saving ---
#     print("\n--- Generating and Saving Plots ---")

#     # Plot predicted class distribution
#     try:
#         if len(predictions) > 0:
#             predicted_classes = pd.Series(predictions).map({0: 'Benign', 1: 'Malware'})
#             predicted_counts = predicted_classes.value_counts()

#             plt.figure(figsize=(8, 6))
#             ax = sns.barplot(x=predicted_counts.index, y=predicted_counts.values, palette='viridis')
#             plt.title('Predicted Class Distribution')
#             plt.xlabel('Predicted Class')
#             plt.ylabel('Count')
#             for container in ax.containers:
#                 ax.bar_label(container)
#             plot1_filename = os.path.join(PLOTS_SAVE_FOLDER, 'predicted_class_distribution.png')
#             plt.savefig(plot1_filename)
#             print(f"✅ Saved predicted class distribution plot to {plot1_filename}")
#             plt.close() # Close the plot figure

#         else:
#             print("No predictions made, skipping predicted class distribution plot.")

#     except Exception as e:
#         print(f"❌ Error generating or saving predicted class distribution plot: {e}")
#         print("Predicted class distribution plotting skipped.")


#     # Plot Confusion Matrix (if available)
#     try:
#         if conf_matrix is not None:
#             plt.figure(figsize=(8, 6))
#             sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
#                         xticklabels=['Predicted Benign', 'Predicted Malware'],
#                         yticklabels=['True Benign', 'True Malware'])
#             plt.title('Confusion Matrix')
#             plt.xlabel('Predicted Label')
#             plt.ylabel('True Label')

#             plot2_filename = os.path.join(PLOTS_SAVE_FOLDER, 'confusion_matrix_plot.png')
#             plt.savefig(plot2_filename)
#             print(f"✅ Saved confusion matrix plot to {plot2_filename}")
#             plt.close() # Close the plot figure

#         else:
#              print("Confusion matrix not available, skipping confusion matrix plot.")


#     except Exception as e:
#         print(f"❌ Error generating or saving confusion matrix plot: {e}")
#         print("Confusion matrix plotting skipped.")


#     print("\nScript finished.")